# Seminario de titulo

## Crear soluciones (no ejecutar si no es necesario)

### Crear Instancias

In [1]:
from clp.get_instance import get_uni

get_uni("instances",10,500)

### Crear soluciones solver

In [ ]:
import sys
sys.path.append("c:/Users/franc/Escritorio/Trabajo-final-de-titulo/src")

from solver.get_solver_solutions import generarSolucionesSolver

generarSolucionesSolver(500)

Ejecutando: wsl /root/proyecto/solver/BSG_CLP2 -i 0 -f BR -t 5 --verbose2=5 /root/proyecto/tests/instances/instances.txt
Ejecutando: wsl /root/proyecto/solver/BSG_CLP2 -i 1 -f BR -t 5 --verbose2=5 /root/proyecto/tests/instances/instances.txt
Ejecutando: wsl /root/proyecto/solver/BSG_CLP2 -i 2 -f BR -t 5 --verbose2=5 /root/proyecto/tests/instances/instances.txt
Ejecutando: wsl /root/proyecto/solver/BSG_CLP2 -i 3 -f BR -t 5 --verbose2=5 /root/proyecto/tests/instances/instances.txt
Ejecutando: wsl /root/proyecto/solver/BSG_CLP2 -i 4 -f BR -t 5 --verbose2=5 /root/proyecto/tests/instances/instances.txt
Ejecutando: wsl /root/proyecto/solver/BSG_CLP2 -i 5 -f BR -t 5 --verbose2=5 /root/proyecto/tests/instances/instances.txt
Ejecutando: wsl /root/proyecto/solver/BSG_CLP2 -i 6 -f BR -t 5 --verbose2=5 /root/proyecto/tests/instances/instances.txt
Ejecutando: wsl /root/proyecto/solver/BSG_CLP2 -i 7 -f BR -t 5 --verbose2=5 /root/proyecto/tests/instances/instances.txt
Ejecutando: wsl /root/proyecto/s

## Reconstruir Bloques

In [1]:
import clp.base
import numpy as np
import math
import clp.box_plotter
import os

from clp.base import Space, BlockList, Block, Aabb
from clp.base import Block, Boxtype, Itemdict

import random

def get_location_block(block,space):
  x,y,z = space.corner_point
  if x == space.xmax: x -= block.l
  if y == space.ymax: y -= block.w
  if z == space.zmax: z -= block.h

  return x,y,z


def transformar_a_bloques(bloques,cajas_por_bloques,lwh2item):
    listaGeneral = []
    bloquesDict = dict()  #diccionario de bloques (n) = bloque[1] -> BlockType(bloque)



    for i in range(len(bloques)):

        bloque = bloques[i].split()
        lwhBloque = [int(valor) for valor in bloque[2].strip('()').split(',')]

        #hacer bl_items para cada bloque -> Diccionario de cajas para cada bloque
        bl_items = Itemdict()

        ##print(cajas_por_bloques[i]) # ['-- box: (49,25,21) n: 12', '-- box: (85,84,72) n: 2', '-- box: (48,36,31) n: 7']
        for j in cajas_por_bloques[i]:
            ##print(j) #-- box: (49,25,21) n: 12
            caja = j.split()
            lwhCaja = [int(valor) for valor in caja[2].strip('()').split(',')]
            n = int(caja[4])
            bl_items[lwh2item[(lwhCaja[0],lwhCaja[1],lwhCaja[2])]] = n


        newBlock = Block(l=lwhBloque[0],w = lwhBloque[1],h = lwhBloque[2],items = bl_items)
        #agregamos el bloque a la lista de bloques validos
        listaGeneral.append(newBlock)
        #agregamos el bloque al diccionario de bloques  [objeto bloque + id]
        #block: 2 (49,21,25)
        blockId = int(bloque[1])
        bloquesDict[blockId] = newBlock

    return listaGeneral, bloquesDict


    ## obtener cajas desde archivo de texto
def obtener_registros_de_tipo_block(archivo):
    try:
        #lista
        bloques = []
        ##lista de listas
        cajas = []
        #abrir archivo
        with open(archivo, 'r') as f:
            #leer cada linea del archivo
            for line in f:
                #verificar si esta linea tiene la palabra "n_blocks:"
                if 'n_blocks:' in line:
                    #almacenamos el numero de cajas que se van a generar
                    n_blocks = int(line.split(':')[1])
                #verificar si esta linea tiene la palabra reservada "-- box:"
                if '-- box:' in line:
                    #asignar esa caja a la misma ubicacion que el bloque
                    cajas[len(bloques) - 1].append(line.strip())

                #verificar si esta linea tiene la palabra reservada "block:"
                if 'block:' in line:
                    #guardamos el bloque en bloques
                    bloques.append(line.strip())
                    #creamos una nueva lista de cajas
                    cajas.append([])
            #los primeros 10000 elementos de bloques son de tipo bloque, los demas son acciones
            f.close()
            return bloques[:n_blocks],cajas,bloques[n_blocks:]
    except FileNotFoundError:
        return f"El archivo '{archivo}' no se encontró."



# generar Space
def genSpace(L,W,H):

    Space.filling = "free"
    Space.vertical_stability = False

    cont = Block(l=L,w=W,h=H)

    space = cont.free_space.closest_space() # espacio libre donde se va a colocar el bloque
    return space,cont


## filtrar los elementos validos para que solo tomemos los que son validos para un space x


## Falta comprobar si funciona -> (recive la lista de bloques y la lista de items que quedan)
def remove_unconstructable(blocks, items):
        #cannot remove while iterating
    to_remove = list()
    for block in blocks:
        if not block.is_constructible(items):
                to_remove.append(block)
    for block in to_remove:
        blocks.remove(block)


    return blocks

def obtenerAcciones(acciones):
    Y = []
    acciones_candidatas = []
    

    for accion in acciones:
        '''selected block:82126 space:(0,0,0)
            action block:78058 eval: 7138616 0.035973469 0.78399454 0.045454545 1401104.8 
            action block:22894 eval: 6283256 0.0088177917 0.76397793 0.071428571 1251514.5 
            action block:22661 eval: 5834452 0.0088177917 0.75365579 0.076923077 1117010 
            action block:80927 eval: 5813328 0.0088177917 0.75365579 0.0625 1067693.2 
            action block:22878 eval: 5385648 0.0088177917 0.74238952 0.083333333 986466.95 
            action block:82126 eval: 6161392 0.012875536 0.61978122 0.055555556 503442.55 '''

        if "selected" in accion:    
            #Selected block
            x,y,z = accion.split()[2].split(":")[1].strip("()").split(",")
            selected_block = int(accion.split()[1].split(":")[1])
            Y.append([selected_block,int(x),int(y),int(z)])
            heuristic_eval = dict()
            acciones_candidatas.append([])

        if "action" in accion:
            block_id = int(accion.split()[1].split(":")[1])
            eval_block = accion.split()[3:]
            eval_block = [float(x) for x in eval_block] #if x.replace('.','',1).isdigit()
            heuristic_eval[block_id] = eval_block
            acciones_candidatas[len(Y) - 1] = heuristic_eval


        
    return Y, acciones_candidatas


def add_block(self, block, x, y, z):
    """
    Coloca un bloque en la posición especificada, si hay espacio disponible.
    Args:
        block (Block): El bloque que se desea colocar.
        x, y, z (int): Coordenadas de la esquina inferior izquierda del bloque.
    """
    # Crear el Aabb del bloque basado en las dimensiones y la posición.
    aabb = Aabb(x, x + block.l, y, y + block.w, z, z + block.h)

    # Verificar si el espacio está libre.
    """if not self.free_space.is_free(aabb):
        raise Exception(f"No hay suficiente espacio libre para colocar el bloque en la posición ({x}, {y}, {z})")"""

    # Colocar el bloque y recortar el espacio libre.
    self.aabbs.append(aabb)
    self.occupied_volume += block.occupied_volume
    self.weight += block.weight
    self.items += block.items
    self.free_space.crop(aabb, block)
    #print(f"Bloque colocado en posición ({x}, {y}, {z})")


Block.add_block = add_block


def funcionActualizadora(space, block, cont, items, blocks, bloquesDict, accion):

    block_loc = get_location_block(block, space) #-> guardar para generar el vector

    cont.add_block(block, accion[1],accion[2],accion[3])  # añadir el bloque al contenedor
    items -= block.items
    blocks = remove_unconstructable(blocks,items) #-> eliminar bloques que no son validos
    cont.free_space.filter(items)
    space = cont.free_space.closest_space()

    return space, block, cont, items, blocks, block_loc




def get_X(space, valid_blocks, bloques_colocados, cont, cant_cajas):
    #almacenamos las dimensiones del contenedor
    lwhSpace = [space.xmax, space.ymax, space.zmax]
    #next_loc = get_location_block(block, space)

    X = []
    #agregamos el space a la lista
    '''space
    - Dimensiones del bloque
    - posicion espacial del bloque
    - cantidad de cajas del bloque
    - si se encuentra colocado
    - si es contenedor'''
    space = [space.xmax,space.ymax,space.zmax,0,0,0,0,0,1] #las dimensiones del space estan normalizadas

    X.append(space)

    #para cada bloque colocado
    for element in bloques_colocados:
        block = element[1]
        block_loc = element[2]
        block_elements = block.items

        #normalizamos las dimensiones
        norm_dims = [block.l , block.w, block.h]

        #normalizamos la ubicacion
        norm_loc = [block_loc[0], block_loc[1], block_loc[2]]

        norm_elements = sum(block_elements.values())

        norm_block = norm_dims + norm_loc + [norm_elements, 1, 0]

        X.append(norm_block)


    #para cada bloque creamos un elemento => [dim normalizadas, ubicacion normalizada (si aun no esta colocado se deja como 0,0,0), si se encuentra colocado (1,0), si correponde al space (1,0), ]
    for block in valid_blocks:
        #normalizamos las dimensiones
        norm_dims = [block.l, block.w , block.h]



        #estos bloques no estan colocados
        norm_loc = [0, 0, 0]

        norm_elements = sum(block.items.values())


        norm_block = norm_dims + norm_loc + [norm_elements, 0, 0]

        X.append(norm_block)



    return X


def genVectorBinario(accion, bloquesDict, valid_blocks, cantidadElementos, cantidadElementosColocados):
    Y = np.zeros(cantidadElementos)

    blockId = accion[0]
    block = bloquesDict.get(blockId, None)

    if block is None:
        raise ValueError(f"El bloque con ID {blockId} no se encontró en bloquesDict.")

    try:
        posicionSinActualizar = valid_blocks.index(block)
    except ValueError:
        raise ValueError(f"El bloque con ID {blockId} no está en valid_blocks.")

    # +1 por el espacio contenedor, +cantidad de colocados
    posicion = posicionSinActualizar + 1 + cantidadElementosColocados
    if posicion >= cantidadElementos:
        raise IndexError(f"Posición {posicion} fuera del rango del vector Y de tamaño {cantidadElementos}.")

    Y[posicion] = 1
    return Y, posicionSinActualizar


def load_instance(filename = "instancia.txt", type="BF", id_instance=0):
    

    with open(filename, "r") as file:
        #discard first line
        file.readline()

        for j in range(1000):
            file.readline()
            L, W, H = [int(x) for x in file.readline().split()]

            #read second line
            n = int(file.readline())
            #read n lines
            items = Itemdict()
            lwh2item = dict()
            for i in range(n):
                id, l, rotx, w, roty, h, rotz, n = [int(x) for x in file.readline().split()]
                item = Boxtype(id, l, w, h, rotx, roty, rot_h=True, weight=1)
                items[item]=n
                lwh2item[(l,w,h)] = item

            if j == id_instance: return  items, L, W, H, lwh2item




def getSample(x_accion, y_accion, posicion, cantidad_bloques_colocados, tamanioMuestra=100):
    # Posición absoluta del bloque correcto dentro de x_accion
    posicion_elemento = posicion + 1 + cantidad_bloques_colocados
    block = x_accion[posicion_elemento]

    # 1. Eliminar el bloque seleccionado de la lista auxiliar
    x_aux = x_accion[:posicion_elemento] + x_accion[posicion_elemento + 1:]

    # 2. Eliminar el espacio y los bloques ya colocados
    x_aux = x_aux[1 + cantidad_bloques_colocados:]

    # 3. Seleccionar aleatoriamente hasta (tamanioMuestra - 1) elementos
    num_samples = min(tamanioMuestra - 1, len(x_aux))
    muestra = random.sample(x_aux, num_samples)

    # 4. Padding si hay menos elementos que tamanioMuestra - 1
    feature_len = len(block)
    while len(muestra) < tamanioMuestra - 1:
        muestra.append([0] * feature_len)

    # 5. Armar muestra final: espacio + colocados + muestra + bloque correcto
    x_sample = x_accion[:1 + cantidad_bloques_colocados] + muestra
    x_sample.append(block)

    # 6. Etiqueta: 1 solo al final
    y_sample = np.zeros(len(x_sample))
    y_sample[-1] = 1

    return np.array(x_sample), np.array(y_sample)

In [2]:
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences  # ← correcto


def generarX_y(id):
    #abrimos el archivo que contiene el resultado del problema
    #tests\resultados_solver\output1.txt
    archivo = r"tests\resultados_solver\output"+str(id+1)+".txt"

    #obtenemos los datos importantes del archivo
    bloques,cajas,acciones = obtener_registros_de_tipo_block(archivo)  ##(acciones)

    #cargamos la instancia original
    items, L, W, H, lwh2item = load_instance(filename = r"tests\instances\instances.txt", type="BF", id_instance=id)

    acciones, acciones_candidatas = obtenerAcciones(acciones)

    #obtenemos la cantidad de cajas
    cant_cajas = sum(items.values())

    #definimos las variables a utilizar
    valid_blocks, bloquesDict = transformar_a_bloques(bloques,cajas,lwh2item)
    space,cont = genSpace(L,W,H)
    bloques_colocados = []
    X = []
    Y = []

    for accion in acciones:
        #print(space)
        if(space == None): break
        x_accion = get_X(space, valid_blocks, bloques_colocados, cont, cant_cajas)
        y_accion, posicion = genVectorBinario(accion, bloquesDict, valid_blocks, len(x_accion), len(bloques_colocados))

        x_sample,y_sample = getSample(x_accion, y_accion, posicion, len(bloques_colocados), 100)
        space, bloque_actual, cont, items, valid_blocks, block_loc = funcionActualizadora (space, valid_blocks[posicion], cont, items, valid_blocks, bloquesDict,accion)


        id = accion[0]
        bloques_colocados.append([id, bloque_actual, block_loc])

        X.append(x_sample)
        Y.append(y_sample)



    #mostrar la solucion
    #box_dims = []
    #for aabb in cont.aabbs:
        #box_dims.append([aabb.xmin,aabb.ymin,aabb.zmin,aabb.xmax,aabb.ymax,aabb.zmax])

    #box_plotter.plot_container([L,W,H], box_dims)
    return X,Y

def generate_data(runs = 100):
    X = []
    Y = []

    Max_blocks = 135

    #generar los datos
    for i in range(runs):
        print(i)
        x,y = generarX_y(i)

        X += x
        Y += y


    X_padded = pad_sequences(X, padding='post',dtype=np.float64)
    Y_padded = pad_sequences(Y, padding='post',dtype=np.float64)

    

    # 3. Normalización Min-Max por característica
    # Flatten temporalmente para normalizar
    max_blocks = X_padded.shape[2]

    X_reshaped = X_padded.reshape(-1, 9)
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X_reshaped)

    # Restaurar forma original
    X_scaled = X_scaled.reshape(X_padded.shape)


    return np.array(X_scaled),np.array(Y_padded)


X, y = generate_data(1)
print(X)

np.savez_compressed("datos_muestras.npz", X=X, Y=y)




KeyboardInterrupt: 

In [ ]:
from clp.base import Block
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences 

def data_gen_encoder(muestra = 500):
    data_encoder = []
    for i in range(muestra):
        #carga de datos
        archivo = r"tests\resultados_solver\output"+str(i+1)+".txt"

        #obtenemos los datos importantes del archivo
        bloques,cajas,acciones = obtener_registros_de_tipo_block(archivo)  ##(acciones)

        # Encoder
        #bloques disponibles (10000 , 5), cajas disponibles (10, 4)(Dict), Contenedor (1, 3) + (restricciones)

        items, L, W, H, lwh2item = load_instance(filename = r"tests\instances\instances.txt", type="BF", id_instance=i)
        cajas_disponibles = []
        contenedor_inicial = [L,W,H] # dimensiones del contenedor
        for caja in items:
            cajas_disponibles.append([caja.l, caja.w, caja.h, items[caja]]) # cajas disponibles

        valid_blocks, bloquesDict = transformar_a_bloques(bloques,cajas,lwh2item)
        bloques_disponibles = []
        for block in valid_blocks:
            bloques_disponibles.append([block.l, block.w, block.h, block.volume, block.occupied_volume])

            

    



        





data_gen_encoder(1)

10000


In [19]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

def data_gen_encoder(muestra=500):
    data_encoder = []
    for i in range(muestra):
        # carga de datos
        archivo = r"tests\resultados_solver\output" + str(i+1) + ".txt"

        # obtenemos los datos importantes del archivo
        bloques, cajas, acciones = obtener_registros_de_tipo_block(archivo)

        # Encoder
        items, L, W, H, lwh2item = load_instance(
            filename=r"tests\instances\instances.txt",
            type="BF",
            id_instance=i
        )
        
        contenedor_inicial = np.array([1, 1, 1], dtype=float)  # dimensiones del contenedor
        
        # -------------------------------
        # Normalización de CAJAS
        # -------------------------------
        cajas_raw = []
        for caja in items:
            cajas_raw.append([caja.l, caja.w, caja.h, items[caja]])  # (l, w, h, cantidad)
        
        cajas_raw = np.array(cajas_raw, dtype=float)
        
        # Normalizamos dimensiones respecto al contenedor
        cajas_raw[:, 0] /= L
        cajas_raw[:, 1] /= W
        cajas_raw[:, 2] /= H
        
        # Normalizamos solo la cantidad (columna 3)
        scaler_cajas = MinMaxScaler()
        cantidades = cajas_raw[:, 3].reshape(-1, 1)  # convertir a 2D
        cajas_raw[:, 3] = scaler_cajas.fit_transform(cantidades).flatten()

        cajas_norm = cajas_raw  # ya está normalizado

        
        # -------------------------------
        # Normalización de BLOQUES
        # -------------------------------
        valid_blocks, bloquesDict = transformar_a_bloques(bloques, cajas, lwh2item)
        
        bloques_raw = []
        for block in valid_blocks:
            bloques_raw.append([block.l, block.w, block.h, block.volume, block.occupied_volume])
        
        bloques_raw = np.array(bloques_raw, dtype=float)

        if len(bloques_raw) > 0:
            # Normalizamos dimensiones con respecto al contenedor
            bloques_raw[:, 0] /= L
            bloques_raw[:, 1] /= W
            bloques_raw[:, 2] /= H

            # volúmenes normalizados respecto al volumen del contenedor
            volumen_contenedor = L * W * H
            bloques_raw[:, 3] /= volumen_contenedor
            bloques_raw[:, 4] /= volumen_contenedor

            bloques_norm = bloques_raw  # ya está normalizado

        else:
            bloques_norm = np.empty((0, 5))

        # -------------------------------
        # Guardamos todo en data_encoder
        # -------------------------------
        data_encoder.append({
            "cajas": cajas_norm,
            "bloques": bloques_norm,
            "contenedor": contenedor_inicial
        })
    
    return data_encoder


Data_encoder = data_gen_encoder(500)

In [20]:
np.savez_compressed("datos_encoder.npz", data=Data_encoder)